In [1]:
from mysql.connector import connect, Error
import os

In [2]:
connect?

Signature: connect(*args: 'Any', **kwargs: 'Any') -> 'Union[PooledMySQLConnection, MySQLConnection, CMySQLConnection]'
Docstring:
Create or get a MySQL connection object.

In its simpliest form, connect() will open a connection to a
MySQL server and return a MySQLConnection object.

When any connection pooling arguments are given, for example pool_name
or pool_size, a pool is created or a previously one is used to return
a PooledMySQLConnection.

Returns MySQLConnection or PooledMySQLConnection.
File:      c:\users\user\appdata\local\programs\python\python311\lib\site-packages\mysql\connector\pooling.py
Type:      function

In [3]:
pass_word = os.environ.get ('mysql_password')

In [4]:
try:
  connection = connect(username="root", password= pass_word, host= 'localhost', database= 'supermarket')
  print('Connection to database successful.')
except Error as err:
  print(err)

Connection to database successful.


In [5]:
category = ('cat101', 'Laundry')
try:
  query = """
  insert into category (category_id, category_name)
  values (%s, %s)
  """
  cursor = connection.cursor()
  cursor.execute(query, category)
  connection.commit()
  print(f"{cursor.rowcount} record(s) created")
except Error as err:
  print(err)


1062 (23000): Duplicate entry 'cat101' for key 'category.PRIMARY'


In [6]:
categories = [('cat100', 'AK47'),
              ('cat99', 'M4'),
              ('cat98', 'SNIPER'),
              ('cat97', 'Pow'),
              ('cat96', 'Laundry')]

In [7]:
category = ('cat101', 'Laundry')
try:
  query = """
  insert into category (category_id, category_name)
  values (%s, %s,%s, %s)
  """
  cursor = connection.cursor()
  cursor.executemany(query, categories)
  connection.commit()
  print(f"{cursor.rowcount} record(s) created")
except Error as err:    
  print(err)

Failed executing the operation; Not enough parameters for the SQL statement


In [8]:
products = [
            ('PRD100', 'Oral B Toothpaste', 'Cat100', 2200, 25),
            ('PRD101', 'Amoeba', 'Cat100', 2200, 25),
            ('PRD102', 'Gangster', 'Cat98', 2200, 25),
            ('PRD103', 'Setup', 'Cat97', 1500, 25),
            ('PRD104', 'Ginger', 'Cat98', 700, 25),
            ('PRD105', 'Sand', 'Cat97', 100, 25),
            ('PRD106', 'Water', 'Cat96', 100, 25),
            ('PRD107', 'Petrol', 'Cat97', 100, 25),
            ('PRD108', 'Jelly Fish', 'Cat96', 100, 25),
            ('PRD109', 'Hammer', 'Cat100', 100, 25)]

In [10]:
try:
  query = """
  insert into products (product_id, product_name, category_id, price, quantity)
  values (%s, %s,%s, %s, %s)
  """
  cursor = connection.cursor()
  cursor.executemany(query, products)
  connection.commit()
  print(f"{cursor.rowcount} records created")

except Error as err:
  print(err)

1062 (23000): Duplicate entry 'PRD100' for key 'products.PRIMARY'


In [11]:
customers = [('cust101', "Yewande Ruth", '09064546633', 'bukkyo@gmail.com', 'Epe, Lagos'),
             ('cust102', "Momoh Dupe", '09076595733', 'bfafao@gmail.com', 'Oloba, Lagos'),
             ('cust103', "Yayi Abu", '09065027633', 'badfadfyo@gmail.com', 'Bruh Shaggi, Lagos'),
             ('cust104', "Femi Enoch", '09076594733', 'buadfasdfkkyo@gmail.com', 'Officer Woos, Lagos'),
             ('cust105', "ebunoluwa", '09343594733', 'buadfasdfkkyo@gmail.com', 'Officer Woos, Lagos'),]

In [12]:
try:
  query = """
  insert into customers (customer_id, customer_name, phone_number, email_address, address)
  values (%s, %s,%s, %s, %s)
  """
  cursor = connection.cursor()
  cursor.executemany(query, customers)
  connection.commit()
  print(f"{cursor.rowcount} records created")
except Error as err:
  print(err)


1062 (23000): Duplicate entry 'cust101' for key 'customers.PRIMARY'


In [13]:
try:
  query = """
  select * from customers
  """
  cursor = connection.cursor(buffered=True)
  cursor.execute(query)
  print(cursor.fetchone())
except Error as err:
  print(err)

('cust101', 'Yewande Ruth', '09064546633', 'bukkyo@gmail.com', 'Epe, Lagos')


In [14]:
query = """
select * from customers where customer_id = %s
"""

cursor = connection.cursor()
cursor.execute (query, ("cust101",))
print(cursor.fetchall())

[('cust101', 'Yewande Ruth', '09064546633', 'bukkyo@gmail.com', 'Epe, Lagos')]


In [15]:
query = """
select * from customers where customer_name like 'y%'
"""

cursor = connection.cursor()
cursor.execute (query)
print(cursor.fetchall())

[('cust101', 'Yewande Ruth', '09064546633', 'bukkyo@gmail.com', 'Epe, Lagos'), ('cust103', 'Yayi Abu', '09065027633', 'badfadfyo@gmail.com', 'Bruh Shaggi, Lagos')]


In [16]:
customer = 'cust101' 

items = [('PRD101', 5), ('PRD102', 2), ('PRD103', 3)]

In [17]:
import datetime 
date = datetime.datetime.now()
date.strftime("%Y-%m-%d %H:%M:%S")

'2023-03-30 12:35:30'

In [23]:
order_total = 0 
prd_total_list = []
try:
  for item in items: 
    p_id, qty_requested = item
    prod_query = """
      select product_id, price, quantity from products where product_id = %s
      """ 

    cursor = connection.cursor(buffered=True)
    cursor.execute(prod_query, (p_id,)) 
    product_info = cursor.fetchone()
    #print(cursor.fetchall())
    product_price = product_info[1]
    available_qty = product_info[2]

    if available_qty >= qty_requested:
      new_qty = available_qty - qty_requested 
      update_query = """
      update products set quantity = %s where product_id = %s
      """
      cursor.execute (update_query, (new_qty, p_id))
      prd_total = product_price * qty_requested
      prd_total_list.append(prd_total)
      order_total += prd_total

      print(prd_total_list)
      print(order_total)
    else :
      print (f"requested quantity for {p_id} not available, only (available_qty) left")
  order_query = """
  insert into orders (order_date, customer_id, total)
  values (%s, %s, %s)
  """
  date = datetime.datetime.now()
  date.strftime("%Y-%m-%d %H:%M:%S")
  cursor.execute (order_query, (date, customer, order_total))

  get_orderid_query = """
  select max(order_id) from orders
  """
  cursor .execute(get_orderid_query)
  order_id = cursor.fetchone()[0]
  
  for i in range(len(prd_total_list)):
    details_query = """
    insert into order_details (order_id, product_id, quantity, product_total)
    values(%s,%s,%s,%s)
    """
    p_id, p_qty = items[i]
    cursor.execute(details_query, (order_id, p_id, p_qty, prd_total_list [i]))
  connection.commit()
except Error as err:
    print(err)
    connection.rollback()


[11000.0]
11000.0
[11000.0, 4400.0]
15400.0
[11000.0, 4400.0, 4500.0]
19900.0
